<a href="https://colab.research.google.com/github/dohnala/VideoPose3D/blob/master/Notebooks/Detectron2%20-%20inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install detectron2

In [1]:
# install dependencies: 
!pip install pyyaml==5.1 pycocotools>=2.0.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

1.6.0+cu101 True
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [2]:
# install detectron2: (Colab has CUDA 10.1 + torch 1.6)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
assert torch.__version__.startswith("1.6")
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html
     |████████████████████████████████| 6.6MB 1.2MB/s 
     |████████████████████████████████| 2.2MB 16.6MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.1.post20200716-cp36-none-any.whl size=42325 sha256=1122fc015aa9a2aa2e30ca75969b38f52c8b07c93430cf1262df003b9e0b04f6
  Stored in directory: /root/.cache/pip/wheels/81/99/f4/42a6bef61c07b3d78dfe6d7ebff259444c4526504cf72378d7
Successfully built fvcore
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: Pillow 7.0.0
    Uninstalling Pillow-7.0.0:
      Successfully uninstalled Pillow-7.0.0


In [2]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

# Install VideoPose3D

In [32]:
!git clone https://github.com/dohnala/VideoPose3D

Cloning into 'VideoPose3D'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 124 (delta 8), reused 17 (delta 7), pack-reused 98
Receiving objects: 100% (124/124), 9.53 MiB | 12.00 MiB/s, done.
Resolving deltas: 100% (45/45), done.


#Download Pretrained Human3.6m Coco Model

In [33]:
!mkdir VideoPose3D/checkpoint
os.chdir('VideoPose3D/checkpoint')
!wget https://dl.fbaipublicfiles.com/video-pose-3d/pretrained_h36m_detectron_coco.bin
os.chdir('../..')

--2020-09-02 10:53:58--  https://dl.fbaipublicfiles.com/video-pose-3d/pretrained_h36m_detectron_coco.bin
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.75.142, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67892577 (65M) [application/octet-stream]
Saving to: ‘pretrained_h36m_detectron_coco.bin’

pretrained_h36m_det 100%[===================>]  64.75M  12.1MB/s    in 6.4s    

2020-09-02 10:54:05 (10.1 MB/s) - ‘pretrained_h36m_detectron_coco.bin’ saved [67892577/67892577]



#Upload video

In [34]:
from google.colab import files

os.chdir('/content')

!rm -rf videos && mkdir videos
!rm -rf output && mkdir output
!rm -rf VideoPose3D/data/detectronoutput && mkdir VideoPose3D/data/detectronoutput
!rm VideoPose3D/video.mp4
!rm VideoPose3D/output.mp4
!rm VideoPose3D/outputfile.npy

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))
  os.rename(fn, "videos/video.mp4")

rm: cannot remove 'VideoPose3D/video.mp4': No such file or directory
rm: cannot remove 'VideoPose3D/output.mp4': No such file or directory
rm: cannot remove 'VideoPose3D/outputfile.npy': No such file or directory


Saving test 17.42.13.mp4 to test 17.42.13.mp4
User uploaded file "test 17.42.13.mp4" with length 340338 bytes


# Compute 2D Coordinates with Detectron2

In [35]:
!python VideoPose3D/inference/infer_video_d2.py \
    --cfg COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml \
    --output-dir output \
    --image-ext mp4 \
    videos

Processing videos/video.mp4
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 -

#Prepare Detectron Output to fit VideoPose3D Input

In [36]:
!cp output/video.mp4.npz VideoPose3D/data/detectronoutput/video.mp4.npz
os.chdir('VideoPose3D/data') # This script must be launched from the "data" directory
!python prepare_data_2d_custom.py -i detectronoutput -o myvideos
os.chdir('../../')

Parsing 2D detections from detectronoutput
Processing detectronoutput/video.mp4.npz
65 total frames processed
0 frames were interpolated
----------
Saving...
Done.


# Compute 3D Joints with VideoPose3D

In [37]:
!cp ./videos/video.mp4 VideoPose3D/video.mp4
os.chdir('VideoPose3D')

!python run.py -d custom -k myvideos -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject video.mp4 --viz-action custom --viz-camera 0 --viz-video video.mp4 --viz-output output.mp4 --viz-export outputfile --viz-size 6


Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='myvideos', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export='outputfile', viz_limit=-1, viz_no_ground_truth=False, viz_output='output.mp4', viz_size=6, viz_skip=0, viz_subject='video.mp4', viz_video='video.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO: Receptive field: 243 frames
INFO: Tr

#Display Results - Joint Export

In [9]:
data  = np.load('outputfile.npy')
lst = data
for item in lst:
    print(item)

[[-1.67904727e-06 -2.67946198e-05  2.19991398e-06]
 [-1.21907458e-01  8.07096157e-03 -6.21215478e-02]
 [ 6.33870736e-02  2.32253343e-01 -4.22959208e-01]
 [ 4.63512391e-02  6.20529413e-01 -2.14335978e-01]
 [ 1.21905506e-01 -8.08252953e-03  6.21319488e-02]
 [ 3.44793618e-01  2.13108450e-01 -2.40165457e-01]
 [ 2.59757251e-01  6.17801428e-01 -9.73307937e-02]
 [ 7.98259377e-02 -1.25853658e-01 -1.90914810e-01]
 [ 2.01983571e-01 -2.01707155e-01 -3.95992488e-01]
 [ 2.57549405e-01 -2.04494596e-01 -4.99740034e-01]
 [ 2.67921150e-01 -3.19034338e-01 -4.92233932e-01]
 [ 3.05483103e-01 -1.51344478e-01 -2.97091067e-01]
 [ 3.75866711e-01  1.12757064e-01 -2.43006587e-01]
 [ 4.03243899e-01  3.51656854e-01 -3.36985350e-01]
 [ 7.10397959e-02 -1.41656920e-01 -4.29523230e-01]
 [-5.23789180e-03  1.34733364e-01 -4.45315838e-01]
 [ 2.59359255e-02  3.82156074e-01 -5.25747418e-01]]
[[-1.68635961e-06 -2.69725351e-05  2.17020533e-06]
 [-1.21833645e-01  8.30822717e-03 -6.19087629e-02]
 [ 6.30094558e-02  2.35268772e

#Display Results - Joint Video

In [38]:
#display video
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

show_local_mp4_video('output.mp4', width=960, height=720)

#Download Joint Export & Video

In [39]:
from google.colab import files


files.download('output.mp4')
files.download('outputfile.npy')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>